In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sb
import os
from matplotlib import rcParams
import pickle

In [3]:
rc = {}
rc['axes.formatter.limits'] = (-2, 3)
rc['axes.labelsize'] = 14
rc['xtick.labelsize'] = 14
rc['ytick.labelsize'] = 14
rc['axes.titlesize'] = 18
rc['figure.titlesize'] = 20
sb.set_style(style='darkgrid', rc=rc)

In [4]:
%matplotlib inline

In [5]:
dataroot = os.getenv('DATADIR')
devroot = os.getenv('DEVDIR')

In [6]:
datadir = os.path.join(dataroot,'LearningKaggle')

In [6]:
files = {fn: os.path.join(datadir, fn) for fn in os.listdir(datadir)}

In [7]:
files

{'item_categories.csv': '/home/madhatter106/DATA/LearningKaggle/item_categories.csv',
 'items.csv': '/home/madhatter106/DATA/LearningKaggle/items.csv',
 'sales_train_v2.csv': '/home/madhatter106/DATA/LearningKaggle/sales_train_v2.csv',
 'sample_submission.csv': '/home/madhatter106/DATA/LearningKaggle/sample_submission.csv',
 'shops.csv': '/home/madhatter106/DATA/LearningKaggle/shops.csv',
 'test.csv': '/home/madhatter106/DATA/LearningKaggle/test.csv'}

In [8]:
df_test = pd.read_csv(files['test.csv'])

In [9]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [10]:
pd.testing.assert_series_equal(df_test.index.to_series(), df_test.ID, check_names=False)

In [11]:
print(f'unique shop id: {len(df_test.shop_id.unique())}')
print(f'unique item id: {len(df_test.item_id.unique())}')

unique shop id: 42
unique item id: 5100


In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [13]:
df_sales_train = pd.read_csv(files['sales_train_v2.csv'])

In [14]:
df_sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [44]:
df_sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
datetime          datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 156.8+ MB


Convert make a datetime object column out of 'date' for access to pandas date/time/timeseries functions

In [15]:
df_sales_train['datetime'] = pd.to_datetime(df_sales_train.date, format='%d.%m.%Y')

Sales prediction done on a monthly basis. Extract month data from datetime column to identify possible cycles/seasonality

In [18]:
df_sales_train['month'] = df_sales_train.datetime.dt.month

In [19]:
df_sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,datetime,month
0,02.01.2013,0,59,22154,999.00,1.0,2013-01-02,1
1,03.01.2013,0,25,2552,899.00,1.0,2013-01-03,1
2,05.01.2013,0,25,2552,899.00,-1.0,2013-01-05,1
3,06.01.2013,0,25,2554,1709.05,1.0,2013-01-06,1
4,15.01.2013,0,25,2555,1099.00,1.0,2013-01-15,1


In [24]:
print("training_set")
print('-'* 50)
print(f'unique date:{len(df_sales_train.datetime.unique())}')
print(f'unique shops: {len(df_sales_train.shop_id.unique())}')
print(f'unique items: {len(df_sales_train.item_id.unique())}', end='\n\n')

print("test set")
print('-'*50)
print(f'uniques shops: {len(df_test.shop_id.unique())}')
print(f'unique items:{len(df_test.item_id.unique())}')

training_set
--------------------------------------------------
unique date:1034
unique shops: 60
unique items: 21807

test set
--------------------------------------------------
uniques shops: 42
unique items:5100


In [26]:
# Are all shops id in test set also contained in train?
set(df_test.shop_id.unique()).issubset(df_sales_train.shop_id.unique())

True

In [27]:
# Are all item id in test set also contained in train set?
set(df_test.item_id.unique()).issubset(df_sales_train.item_id.unique())

False

Weird: some items that are in the test set are not contained in the train set. Investigate that next. But first pickle the data for easy re-call, just in case.

In [30]:
os.makedirs('PklJar')

In [31]:
with open('train_test_data.pkl', 'wb') as f:
    datadict = {'trainset': df_sales_train,
                'testset': df_test,
                'files': files
               }
    pickle.dump(datadict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
# How many items in test that are not in train:
len(set(df_test.item_id.unique()) - set(df_sales_train.item_id.unique()))

363

In [28]:
# which ones?
df_test['item_is_in_train'] = df_test.item_id.isin(df_sales_train.item_id.values)

In [32]:
df_test.head()

,ID,shop_id,item_id,item_is_in_train
0,0,5,5037,True
1,1,5,5320,False
2,2,5,5233,True
3,3,5,5232,True
4,4,5,5268,False


In [34]:
df_test[~df_test.item_is_in_train].item_id.unique()

array([ 5320,  5268,  5826,  3538,  3571,  3604,  3407,  3408,  3405,
        3984,  8551,  8822,  8823,  8824,  8890, 10779,  9767,  8355,
        8354,  6729,  6731,  6732,  6152,  6153,  6742,  7782,  7862,
        6743,  6335, 13559, 13804, 13402, 14647, 14832, 14959,  2327,
        1732,  2322,  2323,  2326,  2569,  2966, 21467, 20400, 20401,
       20836,  1580,  1585,  1246,  1437,  2328,  2427,  2325,  3022,
        1193,  1252,  1386,  1884,  1885,  1577,  3627,  4412,  3271,
        3349, 13558, 13310, 13338, 12890, 13805, 13309, 12568, 13275,
       13251, 13303, 13250, 13249, 13232, 13242, 14648, 19773, 18913,
       21397, 21708, 16051, 16141, 16193, 15553,  1284,  5269,  6730,
        3338,  4060, 11974,  7728,  7729,  7926,  8549,  8826,  5812,
        5025,  9972, 11762,  7669, 13263, 13264,  8993,  7727, 18174,
       19219,  6863, 12117, 13209, 13273, 14521,  7592,  7607,  7646,
        6903,  8545,  5322,  4156,  5843, 11385, 13271, 10028, 10203,
       16139, 22022,